[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/mcda.ipynb)


## Conjunto de dados

Trabalharemos inicialmente com um conjunto de dados do Legathum Prosperity Index que avalia países com base em 12 indicadores socioeconômicos. 


In [ ]:
import pandas as pd # biblioteca utilizada para a leitura de dados
import numpy as np # biblioteca para trabalhar com matrizes e operações de Álgebra Linear em Python

In [ ]:
# A matriz de dados está disponível no link do github abaixo. 
# Assim, a leitura dos dados pode ser realizada de maneira direta através do link

url = 'https://raw.githubusercontent.com/diogoflim/ProjIntegrador_PO_IA/main/Dados/LegathumProsperityIndex.csv'
df = pd.read_csv(url, index_col=0, sep = ';')

In [ ]:
# Visualização inicial da tabela
df

In [ ]:
# O método describe da biblioteca pandas pode ser usado para realizarmos uma descriç~]ao inicial dos dados da tabela 
df.describe()

In [ ]:
# Além disso, podemos plotar a distribuição dos dados para cada um dos critérios. Vejamos abaixo.

df.hist(bins=50, figsize=(15, 10))

Para o uso da biblioteca PyDecision, passaremos a matriz de decisão para o formato de array do numpy.

Isso pode ser realizado de maneira simples, conforme ilustrado abaixo

In [ ]:
X = np.array(df)
X

In [ ]:
m, n = X.shape

print(f'O número de alternativas é: {m}\nO número de critérios é: {n}')

# Decisão Multicritério

A literatura de decisão multicritério apresenta uma vasta quantidade de métodos para a resolução de problemas de decisão que envolvem múltiplos objetivos, muitas vezes conflitantes.

A Biblioteca PyDecision (https://github.com/Valdecy/pyDecision) inclui diversos métodos em Python. 

In [ ]:
# No Google Colab
!pip install pyDecision

In [ ]:
from pyDecision.algorithm import promethee_ii

## PROMETHEE II

Inicialmente, aplicaremos os métodos PROMETHEE I e PROMETHEE II são usados para ordenar alternativas em problemas de análise multicritério.

Como visto na aula, os parâmetros do PROMETHEE II incluem:

- O tipo de função de preferências de cada critério;
- Limiares de indiferença, preferência e veto para cada critério;
- Pesos dos critérios;
- Parâmetro $s$, caso a função de preferência s-curve seja escolhida para algum critério.

In [ ]:
# Parâmetros
Q = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5] # Limiares de indiferença
P = [20 for j in range(n)] # Limiares de preferência
W = [1/n for j in range(n)] # pesos
S = [0 for j in range(n)] # s (caso use o critério Gaussiano)
F = ['t5' for j in range(n)] # 't1' = Usual; 't2' = U-Shape; 't3' = V-Shape; 't4' = Level; 't5' = V-Shape with Indifference; 't6' = Gaussian; 't7' = C-Form


In [ ]:
P

Vamos aplicar o PROMETHEE e armazenar os fluxos líquidos em p2.

Imprimiremos na tela os resultados das 20 primeiras alternativas 

In [ ]:
resultados = promethee_ii(X, W = W, Q = Q, S = S, P = P, F = F, sort = False, topn = 20)

Podemos acessar os fluxos líquidos das alternativas e incluí-los em um DataFrame!

In [ ]:
ranking = pd.DataFrame(resultados[:,1], columns = ['Fluxo Líquido'], index = df.index)
ordem = ranking.sort_values("Fluxo Líquido", ascending=False)
ordem.iloc[:10] #print os 10 primeiros colocados

## Exercício 1

Considere o problema de decisão da ordenação de países com base nos dados do Legatum Institute. 

Ao invés de utilizar a função V-shape (tipo 5) para todos os critérios, poderíamos usar funções diferentes para critérios distintos. 

Além disso, poderíamos usar valores distintos para os limiares de preferência e indiferença!

Faça o seguinte teste:

- Para os oito primeiros critérios, utilize a função de preferência critério usual (tipo 1)
- Para os critérios $g_9$ e $g_{10}$, use a função do tipo 2, com parâmetros: $q_9 = 7$ e $q_{10} = 6$.
- Para os critérios $g_{11}$ e $g_{12}$, use a função do tipo 5, com os parâmetros a seguir: $q_{11} = q_{12} = 10$ e $p_{11} = p_{12} = 20$

O ranking se alterou? Dentre as 10 primeiras alternativas, quantas permaneceram?

## Exercício 2: Ordenação de Fornecedores

Uma organização deseja ordenar suas alternativas de fornecedores para a realização de uma certa atividade. Para isso, seis critérios foram selecionados e as avaliações dos fornecedores é dada na tabela abaixo.

| Fornecedores | Preço         | Qualidade | Confiabilidade | Credibilidade | Assistência | Lead time |
|--------------|---------------|-----------|-----------------|---------------|-------------|-----------|
| Fornecedor 1 | 1.834.600,20  | 1         | 18              | 90            | 1           | 30        |
| Fornecedor 2 | 1.823.240,00  | 2         | 5               | 100           | 0           | 30        |
| Fornecedor 3 | 1.902.311,00  | 3         | 10              | 80            | 1           | 30        |
| Fornecedor 4 | 1.795.200,00  | 1         | 9               | 95            | 0           | 30        |
| Fornecedor 5 | 1.712.000,50  | 2         | 1               | 100           | 0           | 35        |
| Fornecedor 6 | 1.697.890,25  | 3         | 16              | 90            | 1           | 45        |
| Fornecedor 7 | 1.722.765,33  | 2         | 4               | 85            | 1           | 40        |

Fonte: [artigo_sbpo](https://proceedings.science/sbpo/sbpo-2022/trabalhos/uso-do-metodo-fitradeoff-para-selecao-de-fornecedores-em-um-processo-licitatorio?lang=pt-br)

Aplique o método PROMETHEE II para ordenar as alternativas.

Observação: Perceba que para os critérios "preço" e "lead time" valores mais altos são piores. Assim, multiplique essas colunas por -1 antes de aplicar o método.

In [ ]:
data = {
    'Fornecedores': ['Fornecedor 1', 'Fornecedor 2', 'Fornecedor 3', 'Fornecedor 4', 'Fornecedor 5', 'Fornecedor 6', 'Fornecedor 7'],
    'Preço': [-1834600.20, -1823240.00, -1902311.00, -1795200.00, -1712000.50, -1697890.25, -1722765.33],
    'Qualidade': [1, 2, 3, 1, 2, 3, 2],
    'Confiabilidade': [18, 5, 10, 9, 1, 16, 4],
    'Credibilidade': [90, 100, 80, 95, 100, 90, 85],
    'Assistência': [1, 0, 1, 0, 0, 1, 1],
    'Lead time': [-30, -30, -30, -30, -35, -45, -40]
}

df = pd.DataFrame(data).set_index("Fornecedores")
df

1. Inicialmente, escolha a função do tipo 1 para todos os critérios e utilize pesos iguais.



2. Em seguida, no critério confiabilidade, selecione a função do tipo 5 usando os parâmetros $q_3 = 3$ e $p_3 = 5$.


3. Realize variações nos parâmetros para realizar testes adicionais. Por exemplo, para um decisor que prioriza o lead time (cosidera esse critério com peso mais alto que os demais), o resultado seria alterado?
